# Introduction 
In aerospace, we use plot3D files to represent the fluid domain around a geometry. This allows us to perform simulations and calculate flow physics inside of each cell/rectangle. Often the connectivity of these blocks are not easily obtainable. You would have to buy an expensive tool that will create the mesh and output some kind of connectivity file. Sometimes these files are proprietary which doesn't help research. This `plot3d` library in python can be used to find connectivity information and also split the blocks into smaller blocks for evaluation. 

**The goal with this project is to enable research and learning. You shouldn't need to purchase something expensive to calculate all this for you.**


## Plot of Turbine Stage (Stator + Rotor Blocks)
Below shows a plot in paraview of the `StageMesh.xyz` file. Each color represents a rectangular block with 6 sides. I've circled the blocks related to the stator and the blocks related to the rotor. It's important to plot this in paraview because this allows you to split the blocks into `Stator` blocks and `Rotor` blocks later. 

> Note: I am using paraview 5.10.1 to plot. Different versions of paraview may have differences in the way python macros are coded so you may see errors.

![stage_mesh](stagemesh.png)


## Objectives of this Notebook
1. Split the blocks and find connectivity
2. Identify surfaces that should be marked as solid
3. Identify the mixing plane location 

![mixing plane](stage_mixing_plane.png)

In [17]:
# Imports
import sys
from typing import List
from plot3d import read_plot3D, connectivity_fast, rotated_periodicity, write_plot3D, Direction, split_blocks
import os, pickle
import numpy as np
import json 

## Sanity Check Finding negative volumes. 
This should be done before doing anything else. A negative volume will certainly crash the solver.

In [18]:
mesh_filename = 'StageMesh.xyz'
print("Reading mesh")
blocks = read_plot3D(mesh_filename,binary=True,big_endian=False)

# This part checks for negative volumes 
lines = list() 
negative_volumes = False

# for i in range(len(blocks)):
#     v = blocks[i].cell_volumes()
#     if np.min(v)<0:
#         negative_volumes = True
#         lines.append(f'negative volume found in block number: {i} \n')
#         print(lines[-1])

# if negative_volumes:
#     with open('negative_volume.txt', 'w') as f:
#         for line in lines:
#             f.write(line)
            

Reading mesh


## Creating the mesh for stator and rotor
In this step, what I did was plot the entire geometry in paraview then deactivated the blocks to figure out which blocks belonged to the stator and rotor.
Blocks 0 to 3 are for the stator and the rest are the rotor. The code below splits the blocks and exports stator and rotor separately. 

In [19]:
stator_blocks = [blocks[i] for i in range(0,4)]
rotor_blocks = [blocks[i] for i in range(4,len(blocks))]
write_plot3D('stator.xyz',stator_blocks,binary=True)
write_plot3D('rotor.xyz',rotor_blocks,binary=True)
stator_blocks_split = split_blocks(stator_blocks,500000, direction=Direction.i) # Splits the blocks while keeping the gcd. 380,000 is a rough number that it tries to match
rotor_blocks_split = split_blocks(rotor_blocks,500000, direction=Direction.i)
write_plot3D('stator_split.xyz',stator_blocks_split,binary=True)
write_plot3D('rotor_split.xyz',rotor_blocks_split,binary=True)

## Step 1: Split the blocks and find the connectivity
Splitting the blocks into smaller blocks can help the solver run much faster. To improve the time it takes to split and find connectivity/periodicity. You need to ensure each block of your original mesh is divisible by at least 4. The higher the greatest common divisor (gcd), the faster you get the connectivity information 

In [20]:
def find_connectivity(filename:str,nblades:int):
    blocks = read_plot3D(f'{filename}.xyz',binary=True,big_endian=False)
    # Finds the connectivity 
    if not os.path.exists(f'{filename}_connectivity.pickle'):
        print('checking connectivity')
        face_matches, outer_faces_formatted = connectivity_fast(blocks)
        with open(f'{filename}_connectivity.pickle','wb') as f:
            [m.pop('match',None) for m in face_matches] # Remove the dataframe
            pickle.dump({"face_matches":face_matches, "outer_faces":outer_faces_formatted},f)


    # Finds the periodicity once connectivity is found   
    with open(f'{filename}_connectivity.pickle','rb') as f:
        data = pickle.load(f)
        face_matches = data['face_matches']
        outer_faces = data['outer_faces']
    
    print("Find periodicity")
    rotation_angle = 360.0/nblades 
    periodic_surfaces, outer_faces_to_keep,periodic_faces,outer_faces = rotated_periodicity(blocks,face_matches,outer_faces,rotation_axis='x',rotation_angle=rotation_angle)

    with open(f'{filename}_connectivity_periodicity.pickle','wb') as f:
        [m.pop('match',None) for m in face_matches] # Remove the dataframe
        pickle.dump({
            "face_matches":face_matches,
            "periodic_faces":periodic_surfaces,
            "outer_faces":outer_faces_to_keep       
            },f)

In [21]:
find_connectivity(filename="stator_split",nblades=55)
find_connectivity(filename="rotor_split",nblades=60)

checking connectivity
gcd to use 4


Checking connections block 12 with 13: 100%|██████████| 91/91 [02:20<00:00,  1.54s/it]


Find periodicity


Checking connections block 8 with 5: 100%|██████████| 2550/2550 [00:59<00:00, 43.03it/s]   


checking connectivity
gcd to use 4


Checking connections block 22 with 23: 100%|██████████| 276/276 [06:14<00:00,  1.36s/it]


Find periodicity


Checking connections block 2 with 11: 100%|██████████| 5700/5700 [02:08<00:00, 44.36it/s]  


## Plotting the Connectivity Information in Paraview
This tutorial was tested using Paraview 5.10.1. Different versions of paraview may not work. They keep changing things in the pvpython library that breaks compatibility with earlier versions. If there's a major update to paraview that you require support for for example version 6 or 5.20.1 then file a github issue and I'll take a look at it; but if it's 5.11.1 then no, because it's a minor update. Also feel free to submit fixes to pv_library.py for newer versions of paraview. 

There's 2 files that you need for plotting with paraview `pv_library.py` and `pv_stator_plot.py`. You will have to call paraview executable from within this directory using command prompt(windows)

[![Watch the video](https://img.youtube.com/vi/bgSSJQolR0E/default.jpg)](https://www.youtube.com/watch?v=bgSSJQolR0E)



1. Open command prompt in the directory containing `pv_stator_plot.py` and the `.xyz files`
2. Find the path to paraview. For me it's this path: `"C:\Program Files\ParaView 5.10.1-Windows-Python3.9-msvc2017-AMD64\bin\paraview.exe" --script=pv_stator_plot.py`
3. Hit [Enter] and this should load paraview and plot the connectivities from the pickle file. 

You can view the contents of `pv_stator_plot.py` to see how the connectivity information is structured. 

![stator paraview](stator_paraview.png)

## Step 2: Identifying key surfaces 
Now that the data is in paraview you can select the surfaces and identify which ones are the blade and mixing plane. 

**Blade surface** I use paraview to help identify the block and the surface (IMIN, JMIN, KMIN) In the code below you'll see where I've identified the stator_body, hub, and mixing plane. I do the same for the rotor. 

In [22]:
from plot3d import Face, find_connected_face,find_face,read_plot3D
import pickle
import numpy as np 
# Stator 
blocks = read_plot3D('stator_split.xyz',binary=True)
with open('stator_split_connectivity_periodicity.pickle','rb') as f:
    data = pickle.load(f)
    face_matches = data['face_matches']
    outer_faces = data['outer_faces']

# Stator body
block_id = 14; indices = np.array([0,0,0,52,148,0], dtype=int) # this is the face we need to find matches for
stator_face_to_match = find_face(blocks,block_id, indices,outer_faces)
stator_faces,outer_faces = find_connected_face(blocks,stator_face_to_match, outer_faces)
stator_faces.append(stator_face_to_match.to_dict())

# Stator Hub
block_id = 0; indices = np.array([0,0,0,32,0,76],dtype=int)
stator_hub_to_match = find_face(blocks,block_id, indices,outer_faces)
stator_hub, outer_faces = find_connected_face(blocks,stator_hub_to_match, outer_faces)
stator_hub.append(stator_hub_to_match.to_dict())


# Stator Shroud
block_id = 0; indices = np.array([0,148,0,32,148,76],dtype=int)
stator_shroud_to_match = find_face(blocks,block_id, indices,outer_faces)
stator_shroud, outer_faces = find_connected_face(blocks,stator_shroud_to_match, outer_faces)
stator_shroud.append(stator_shroud_to_match.to_dict())

# Mixing Plane
block_id = 9; indices = np.array([36, 0,0, 36,148,36], dtype=int)
mixing_plane_face_to_match = find_face(blocks,block_id, indices,outer_faces)
stator_mixing_plane_faces, outer_faces = find_connected_face(blocks,stator_face_to_match, outer_faces)
stator_mixing_plane_faces.append(stator_face_to_match.to_dict())

data['outer_faces'] = outer_faces
data['mixing_plane'] = stator_mixing_plane_faces
data['stator_body'] = stator_faces
data['stator_shroud'] = stator_shroud
data['stator_hub'] = stator_hub

with open('stator_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)


AttributeError: 'NoneType' object has no attribute 'BlockIndex'

In [ ]:
# Rotor
blocks = read_plot3D('rotor_split.xyz',binary=True)
with open('rotor_split_connectivity_periodicity.pickle','rb') as f:
    data = pickle.load(f)
    face_matches = data['face_matches']
    outer_faces = data['outer_faces']

# Mixing Plane
block_id = 0; indices = np.array([0,0,0,0,148,76], dtype=int) # this is the face we need to find matches for
rotor_mixing_plane_face_to_match = find_face(blocks,block_id, indices,outer_faces)
rotor_mixing_plane_faces, outer_faces = find_connected_face(blocks,rotor_mixing_plane_face_to_match, outer_faces)# Should match with block 10 [0,0,0,0,148,52]
rotor_mixing_plane_faces.append(rotor_mixing_plane_face_to_match.to_dict())  

# Rotor body
block_id = 15; indices = np.array([0,0,0,52,148,0],dtype=int)
rotor_face_to_match = find_face(blocks,block_id, indices,outer_faces)
rotor_faces, outer_faces = find_connected_face(blocks,rotor_mixing_plane_face_to_match, outer_faces)
rotor_faces.append(rotor_face_to_match.to_dict())

# Rotor Hub
block_id = 0; indices = np.array([0,148,0,32,148,76],dtype=int)
rotor_hub_to_match = find_face(blocks,block_id, indices,outer_faces)
rotor_hub, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_hub.append(rotor_hub_to_match.to_dict())

# Rotor Shroud
block_id = 0; indices = np.array([0,0,0,32,0,76],dtype=int)
rotor_shroud_to_match = find_face(blocks,block_id, indices,outer_faces)
rotor_shroud, outer_faces = find_connected_face(blocks,rotor_hub_to_match, outer_faces)
rotor_shroud.append(rotor_shroud_to_match.to_dict())

data['outer_faces'] = outer_faces
data['mixing_plane'] = rotor_mixing_plane_faces
data['rotor_body'] = rotor_faces
data['rotor_hub'] = rotor_hub
data['rotor_shroud'] = rotor_shroud

with open('rotor_split_connectivity_final.pickle','wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# Putting it all together
The code below combines the connectivity, periodicity, surface information and exports it to a JSON file. 